In [1]:
import ee
import pandas as pd

In [2]:
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=C7jekviIj9jM8BkcFo48DHaagLogzsBmaRCK_GmrtNo&tc=VQuWoVRUBAMvuMnySJxeA3HjZOGuT_y5WEvjnMwrv1Q&cc=fUJPC8QJcpmtFUYPFHq_QelVPXqeONss4cJnUzwhz_E

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AWtgzh4jPUmgFsT_PVOwU_UlshRPcMUpOpe-BjghPf8i_d_zxaZJgT-LhFA

Successfully saved authorization token.


In [3]:
geometry = ee.Geometry.Polygon([
                          [[77.39051195702437,23.26465625526173],
                           [77.39041673860434,23.26412338843017],
                           [77.3904636772621,23.26405624089797],
                           [77.39118921480063,23.26394658705701],
                           [77.39125761113051,23.263991557463306],
                           [77.3913186313856,23.264344543824414],
                           [77.39139239213351,23.264538593519962],
                           [77.39151242098693,23.264640854515545],
                           [77.39166731855754,23.264705537634978],
                           [77.39190201184634,23.264666727767032],
                           [77.39205422720794,23.264503479786086],
                           [77.39206945562391,23.264350538413026],
                           [77.39255761766462,23.26425751766926],
                           [77.39281913304357,23.264235956560427],
                           [77.39302968645124,23.264174353373196],
                           [77.39314099812536,23.264049914848027],
                           [77.39321073555975,23.263911923476428],
                           [77.3932375576499,23.263718488902498],
                           [77.39351650738745,23.263754218876777],
                           [77.39349504971533,23.264226100052102],
                           [77.39335959816007,23.26531647183388],
                           [77.39309808278112,23.26623681259786],
                           [77.39302566313772,23.26630087892045],
                           [77.39269441032438,23.266314431407825],
                           [77.39194607400923,23.26627500598619],
                           [77.3916349377635,23.266222028057506],
                           [77.39142974877386,23.266160425788357],
                           [77.3912567462924,23.26613208873483],
                           [77.39100193643598,23.266026132742766],
                           [77.39097645545034,23.265981779046644],
                           [77.39088257813482,23.26584871786952],
                           [77.39083295726805,23.265785883378683],
                           [77.39070823454885,23.26540025144257],
                           [77.39063117839923,23.26505848385452],
                           [77.39060435630908,23.26490324472629],
                           [77.39056412317386,23.264692562763003],
                           [77.39051195702437,23.26465625526173]]])

## Sentinel 2 params

In [16]:
import numpy as np
import pandas as pd

start_date = '2020-01-01'
end_date = '2022-12-31'

def param_data(start_date, end_date):
  ndwi_bands = ['B3', 'B8']
  ndti_bands = ['B8A', 'B12']
  ndci_bands = ['B4', 'B3']
  do_bands = ['B4', 'B5']

  sentinel = ee.ImageCollection('COPERNICUS/S2_SR') \
                .filterBounds(geometry) \
                .filterDate(start_date, end_date) \
                .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
                .median() 
  

  # NDWI data
  
  ndwi = sentinel.normalizedDifference(ndwi_bands).rename('ndwi')

  latlon = ee.Image.pixelLonLat().addBands(ndwi)

  latlon = latlon.reduceRegion(
      reducer=ee.Reducer.toList(),
      geometry=geometry,
      scale=100)

  ndwi_data = np.array((ee.Array(latlon.get('ndwi')).getInfo()))

  # NDTI data

  ndti = sentinel.normalizedDifference(ndti_bands).rename('ndti')

  latlon = ee.Image.pixelLonLat().addBands(ndti)

  latlon = latlon.reduceRegion(
      reducer=ee.Reducer.toList(),
      geometry=geometry,
      scale=100)
  
  ndti_data = np.array((ee.Array(latlon.get('ndti')).getInfo()))


  # NDCI data

  ndci = sentinel.normalizedDifference(ndci_bands).rename('ndci')

  latlon = ee.Image.pixelLonLat().addBands(ndci)

  latlon = latlon.reduceRegion(
      reducer=ee.Reducer.toList(),
      geometry=geometry,
      scale=100)
  
  assert len(ndci_bands) == 2, 'bands_ndci should contain exactly 2 band names'

  ndci_data = np.array((ee.Array(latlon.get('ndci')).getInfo()))


  # Dissolved oxygen data

  do = sentinel.normalizedDifference(do_bands).rename('do')

  latlon = ee.Image.pixelLonLat().addBands(do)

  latlon = latlon.reduceRegion(
      reducer=ee.Reducer.toList(),
      geometry=geometry,
      scale=100,
      tileScale=16)
  
  do_data = np.array((ee.Array(latlon.get('do')).getInfo()))

  # Dataframe

  data = pd.DataFrame({'NDWI': ndwi_data,
                         'NDTI': ndti_data,
                         'NDCI':ndci_data,
                         'DO':do_data})
  
  return data
  
data = param_data(start_date, end_date)
print(data)

       NDWI      NDTI      NDCI        DO
0 -0.151365  0.087177 -0.034686 -0.168233
1  0.097406  0.248638 -0.185957 -0.270476
2  0.092679  0.252130 -0.185777 -0.281252
3  0.096341  0.261943 -0.201069 -0.292475
4  0.039835  0.268859 -0.179384 -0.265648
5  0.102881  0.239123 -0.197415 -0.287364
6  0.025927  0.230534 -0.175719 -0.261873
